In [5]:
from pysparkling import Context

Word Count
---------

In [6]:
counts = Context().textFile(
    '../README.rst'
).map(
    lambda line: ''.join(ch if ch.isalnum() else ' ' for ch in line)
).flatMap(
    lambda line: line.split(' ')
).map(
    lambda word: (word, 1)
).reduceByKey(
    lambda a, b: a + b
)
print(counts.collect())

[(u'', 1426), (u'05', 1), (u'1', 2), (u'100k', 1), (u'11', 2), (u'2015', 4), (u'46333', 1), (u'A', 1), (u'ACCESS', 2), (u'API', 3), (u'AWS', 3), (u'Broadcast', 2), (u'BytesIO', 1), (u'CAACGCCGCGTGAGGGATGACGGCCTTCGGGTTGTAAACCTCTTTCAGTATCGACGAAGC', 1), (u'CC', 2), (u'Common', 2), (u'Context', 8), (u'Count', 1), (u'Crawl', 2), (u'DEMO', 1), (u'Examples', 1), (u'False', 2), (u'Features', 1), (u'File', 1), (u'Get', 1), (u'HISTORY', 2), (u'HM16STR', 1), (u'Hadoop', 1), (u'Handles', 1), (u'Human', 1), (u'ID', 1), (u'If', 1), (u'Infers', 1), (u'Install', 1), (u'It', 1), (u'JVM', 2), (u'KEY', 2), (u'MAIN', 2), (u'Microbiome', 1), (u'More', 1), (u'None', 7), (u'Normally', 1), (u'Now', 1), (u'Parallelization', 1), (u'Pool', 3), (u'Project', 1), (u'PySpark', 1), (u'Python', 1), (u'RDD', 20), (u'RDDs', 1), (u'README', 1), (u'Resolves', 1), (u'S3', 5), (u'SECRET', 1), (u'Spark', 2), (u'SparkContext', 1), (u'Specify', 1), (u'StatCounter', 1), (u'Supports', 1), (u'The', 6), (u'They', 1), (u'This', 2),

Accessing Public Datasets on AWS S3
===================

You need to have valid AWS account credentials in your environment under ``AWS_ACCESS_KEY_ID`` and ``AWS_SECRET_ACCESS_KEY`` for the next demos.


Common Crawl
-----------

In [7]:
# read all the paths of warc and wat files of the latest Common Crawl
paths_rdd = Context().textFile(
    's3n://aws-publicdatasets/common-crawl/crawl-data/CC-MAIN-2015-11/warc.paths.*,'
    's3n://aws-publicdatasets/common-crawl/crawl-data/CC-MAIN-2015-11/wat.paths.gz'
)

print(paths_rdd.takeSample(5))

[u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936464303.77/warc/CC-MAIN-20150226074104-00038-ip-10-28-5-156.ec2.internal.warc.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936462426.4/wat/CC-MAIN-20150226074102-00080-ip-10-28-5-156.ec2.internal.warc.wat.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936463093.76/wat/CC-MAIN-20150226074103-00303-ip-10-28-5-156.ec2.internal.warc.wat.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424936463425.63/wat/CC-MAIN-20150226074103-00312-ip-10-28-5-156.ec2.internal.warc.wat.gz', u'common-crawl/crawl-data/CC-MAIN-2015-11/segments/1424937406179.50/wat/CC-MAIN-20150226075646-00231-ip-10-28-5-156.ec2.internal.warc.wat.gz']


Human Biome Project
------------------

In [8]:
# access data from the Human Microbiome Project
by_subject_rdd = Context().textFile(
    's3n://human-microbiome-project/DEMO/HM16STR/46333/by_subject/*'
)
print(by_subject_rdd.takeSample(1))

[u'GCGCTCAGAGATGGGTGTGCCTCTTTTGGGTTCGGTTCACAGGTGGTGCATGGCTGTCGT']
